In [ ]:
import torch
from preprocessing import get_dataframes
from dataloading import Text_dataset, get_dataloaders
from utilities import set_seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

# Enforce reproducability
set_seed(42)

# Create dataframes
train_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/length_split/tasks_train_length.txt"
test_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/length_split/tasks_test_length.txt"
train_df, test_df, voc_in, voc_out = get_dataframes(train_url, test_url)

# Create datasets
train_data = Text_dataset(train_df[['IN_idx', 'OUT_idx']], sample=True, size=100000)
test_data = Text_dataset(test_df[['IN_idx', 'OUT_idx']], sample=False)

# Create
train_loader, test_loader= get_dataloaders(train_data, test_data, 4)

In [ ]:
from lightning import Trainer
import models

# Best in experiment 2
rnn = 'gru'
layers = 1
hidden_size = 50
dropout = 0.5
max_len = train_df.OUT_idx.apply(len).max()

encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout)
decoder = models.AttnDecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len, dropout)
seq2seq = models.Seq2SeqModel(encoder, decoder, len(train_loader))
trainer = Trainer(max_epochs=1)
print("Fitting model..")
trainer.fit(model=seq2seq, train_dataloaders=train_loader)

In [ ]:
import models
from training import train

# Best in experiment 2
rnn = 'gru'
layers = 1
hidden_size = 50
dropout = 0.5
max_len = train_df.OUT_idx.apply(len).max()

for i in range(5):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.AttnDecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len, dropout).to(device)      
    train(train_loader, encoder, decoder, device, save_name='ex2_best_'+str(i))

In [ ]:
import models
from training import train

# Overall best
rnn = 'lstm'
layers = 2
hidden_size = 200
dropout = 0.5
max_len = train_df.OUT_idx.apply(len).max()

for i in range(5):
    set_seed(i)
    encoder = models.EncoderRNN(rnn, len(voc_in), hidden_size, layers, dropout).to(device)
    decoder = models.DecoderRNN(rnn, hidden_size, len(voc_out), layers, device, max_len).to(device)      
    train(train_loader, encoder, decoder, device, save_name='ex1_overall_'+str(i))